In [ ]:
import os
import re
import sys
from pprint import pprint

import keyring
from imap_tools import A, MailBox, O

sys.path.append('..')
sys.path.append('../..')
import pipeline
from MongoDB_connect import mongodb_connect

In [ ]:
def build_mailbox_dict(folder, mapping, dry_run=False, debug=False):
    mail_dicts = []
    with MailBox('imap.gmail.com').login(addr, mail_p, initial_folder=folder) as mailbox:
        mails = mailbox.fetch(criteria=A(O(*mapping)), bulk=True)
        for msg in mails:
            include = [
                'date', 'from_', 'reply_to', 'to', 'subject', 'text', 'flags',
                'size', 'uid'
            ]
            msg_ = [(x, getattr(msg, x)) for x in dir(msg) if x in include]
            data = {k: v for k, v in msg_}
            for k, v in msg_:
                try:
                    if len(v) == 1:
                        data.update({k: v[0]})
                except TypeError:
                    pass

            mail_regex = r'On\s[a-zA-Z]+,\s[a-zA-Z]+\s+[0-9]+,\s[0-9]+'
            gmail_regex = r'-+\s[a-zA-Z]+\s+[a-zA-Z]+\s-+'
            content = re.split(mail_regex, data['text'])
            if len(content) == 1:
                content = re.split(gmail_regex, data['text'])

            body = []
            for line in content[0].split('\n'):
                if not any([
                        line.startswith(x)
                        for x in ['From:', 'Subject:', 'Message Body:']
                ]):
                    body.append(line)

            data['text'] = '\n'.join(body)
            data = {k: data[k] for k in include}
            mail_dicts.append(data)
            if folder == '[Gmail]/Sent Mail':
                collection = 'sent_emails'
            elif folder.upper() == 'INBOX':
                collection = 'inbox_emails'
            if not dry_run:
                db[collection].update_one(
                    {'_id': data['uid']},
                    {'$set': data}, upsert=True
                )
                mailbox.box.uid('STORE', data['uid'], 'X-GM-LABELS',
                                'cat-tracker-2.0')
            if debug:
                return data
    return mail_dicts

In [ ]:
session = pipeline.main(True)
db = session.connection['cattracker2']

In [ ]:
addr = keyring.get_password('emails', 'ncsu')
mail_p = keyring.get_password('ncsu.gmail', addr)

In [ ]:
contacted_us = list(
    set(x['reply_to'].lower()
        for x in db.contact_us.find({'reply_to': {
            '$ne': None
        }})))

emails_mapping = list(
    map(lambda x: A(from_=x), contacted_us + ['wordpress@cattracker.org']))
sent_emails_mapping = list(map(lambda x: A(to=x), contacted_us))

In [ ]:
sent_mail_dicts = build_mailbox_dict('[Gmail]/Sent Mail', sent_emails_mapping)

In [ ]:
emails_ = list(db.inbox_emails.find(
    {
        'ignore': {'$ne': True},
        'colab': {'$ne': True}
    }
)
    )
inbox_emails = []
for x in emails_:
    if not x['reply_to']:
        inbox_emails.append(x['from_'])
    else:
        inbox_emails.append(x['reply_to'])
inbox_emails = set(inbox_emails)

diff = list(inbox_emails.difference([x['to'] for x in sent_mail_dicts]))

print('-' * 20, 'HAVE NOT RESPONDED TO', '-' * 20)
for n, m in enumerate(diff):
    print(f'({n})\n')
    res = db.inbox_emails.find({'$or': [{'from_': m}, {'reply_to': m}]})
    m_ = list(res)[0]
    m_['text'] = m_['text'].replace('\r\n', '')
    pprint(m_)
    print('\n' + '-' * 80 + '\n')

In [ ]:
# key = 'ignore'
# key = 'colab'
# if diff:
#     for ignore_num in [0]:
#         db.inbox_emails.update_one(
#             {'$or': [{'from_': diff[ignore_num]}, {'reply_to': diff[ignore_num]}]},
#             {'$set':{key: True}})

Hi <NAME>,

Thank you for your contacting us! We are more than happy to include your feline friend in our research!

You can start by filling out the signup form (https://cattracker.org/signup).  We will be waiting for your form response before we proceed any further with including your cat in the study.

Please let me know if you have any questions!  Thank you again!


Best wishes,
<FROM>